In [23]:
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer
import re

In [27]:
df_100 = pd.read_csv("../result tables/test 1 (1).csv", encoding="latin1")

In [11]:
index = faiss.read_index("E:\\Meliha-Patenti\\index\\faiss_index_ivf_cosine.bin")
metadata = pd.read_csv("E:\\Meliha-Patenti\\metadata_all.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_20268\124690261.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("E:\\Meliha-Patenti\\metadata_all.csv")


In [28]:
df_100 = df_100[['Publication Number']]


print(df_100.columns)

Index(['Publication Number'], dtype='object')


In [21]:
def extract_pub_number_core(pub_num):
    """
    Remove first 2 letters, remove last letter and everything after it.
    Example: 'US20230012345A1' -> '20230012345'
    """
    if pd.isna(pub_num): return None
    s = str(pub_num).strip().upper()
    
    # Remove the first 2 letters (country code)
    if len(s) < 5: return None
    s = s[2:]

    # Remove everything from the last letter onward (A1, B2, etc.)
    last_letter_match = re.search(r"[A-Z](?!.*[A-Z])", s)
    if last_letter_match:
        last_letter_idx = last_letter_match.start()
        s = s[:last_letter_idx]

    return s if s.isdigit() else re.sub(r"\D", "", s)


In [39]:
df_100["patent_id"] = df_100["Publication Number"].apply(extract_pub_number_core).astype(str)
metadata['patent_id'] = metadata['patent_id'].astype(str)

In [40]:
joined = df_100.merge(
    metadata[["patent_id", "cleaned_claim"]],
    on="patent_id",
    how="left",
    validate="one_to_many"
)

In [42]:
joined.to_csv('top100_with_claims.csv', index=False)

In [45]:
joined

,Publication Number,patent_id,cleaned_claim
0,US8442755,8442755,1. A system for a motor vehicle including a mo...
1,US8442755,8442755,11. A system for a motor vehicle including a m...
2,US8442755,8442755,17. A method carried-out by a vehicular system...
3,US7712819,7712819,5. A door for a motor vehicle comprising: a su...
4,US7712819,7712819,1. A door for a motor vehicle comprising: a su...
...,...,...,...
205,US5865496,5865496,44. A door structure of a motor vehicle having...
206,US5865496,5865496,33. A door structure of a motor vehicle having...
207,US5865496,5865496,1. A door structure of a motor vehicle having ...
208,US20120036780,20120036780,NaN


In [46]:
query_claim = joined.loc[0, 'cleaned_claim']

query_claim

'1. A system for a motor vehicle including a motor vehicle side door and an outside rear view mirror, the system comprising: a door motor coupled to the motor vehicle side door and configured to selectively prevent movement of the motor vehicle side door toward a fully open position; a camera coupled to the outside rear view mirror and positioned to monitor an area external to the vehicle and adjacent the motor vehicle side door, the camera configured to determine a first distance between the motor vehicle side door and an obstacle detected within the monitored area; a motor controller operatively coupled to the door motor and configured to selectively disable the motor vehicle side door by commanding the door motor to prevent movement of the motor vehicle side door toward the fully open position; and a processor operatively coupled to the camera and to the motor controller, the processor configured to transmit a signal to the motor controller to disable the motor vehicle side door if 

In [ ]:
model = SentenceTransformer("..\\local_minilm_l6_v2")

q_vec = model.encode([query_claim], normalize_embeddings=True).astype("float32")

# ========================
# SEARCH ENTIRE INDEX
# ========================
print("🔍 Searching entire FAISS index...")
distances, indices = index.search(q_vec, index.ntotal)

# Flatten
indices = indices[0]
distances = distances[0]

# ========================
# MAP BACK TO METADATA
# ========================
results = metadata.iloc[indices].copy()
results["similarity"] = distances

# Optional: normalize patent_id for matching
def normalize_id(x):
    if pd.isna(x): return None
    s = str(x).upper().strip()
    s = re.sub(r"[^A-Z0-9]", "", s)
    s = re.sub(r"^([A-Z]{2,3})0*", "", s)
    s = re.sub(r"[A-Z].*$", "", s)
    return s

metadata["norm_id"] = metadata["patent_id"].apply(normalize_id)
joined["norm_id"] = df_100["patent_id"].apply(normalize_id)
results["norm_id"] = results["patent_id"].apply(normalize_id)

# ========================
# FILTER TO ONLY TOP100 PATENTS
# ========================
results_filtered = results[results["norm_id"].isin(joined["norm_id"])]

# ========================
# SAVE RESULTS
# ========================
results_filtered = results_filtered.sort_values("similarity", ascending=False).reset_index(drop=True)
results_filtered.to_csv("similarities_first_vs_top100.csv", index=False)

print(f"✅ Done. Saved similarities to similarities_first_vs_top100.csv")
print(results_filtered.head(10)[["patent_id", "similarity", 'cleaned_claim']])

🔍 Searching entire FAISS index...
✅ Done. Saved similarities to similarities_first_vs_top100.csv


NameError: name 'CLAIM_COLUMN' is not defined

In [51]:
df_100 = joined

In [52]:
top100_ids = set(df_100["norm_id"].dropna().astype(str))
metadata_ids = set(metadata["norm_id"].dropna().astype(str))

missing_ids = sorted(top100_ids - metadata_ids)

print(f"Unique norm_id in top100: {len(top100_ids)}")
print(f"Unique norm_id in metadata: {len(metadata_ids)}")
print(f"IDs in top100 not found in metadata: {len(missing_ids)}")
print(missing_ids[:20])  # first 20


Unique norm_id in top100: 100
Unique norm_id in metadata: 8328376
IDs in top100 not found in metadata: 40
['20030101781', '20030218356', '20030230910', '20040055110', '20040113456', '20040201277', '20050057047', '20050085972', '20060055184', '20060249982', '20060254142', '20070107314', '20070136987', '20070146120', '20070175099', '20080030045', '20080122257', '20080217956', '20080246294', '20080258495']


In [54]:
CLAIM_COL = 'cleaned_claim'

In [56]:
from sklearn.metrics.pairwise import cosine_similarity

# Make sure CLAIM_COL is all strings and no NaNs
joined = joined[joined[CLAIM_COL].notna()].reset_index(drop=True)
joined[CLAIM_COL] = joined[CLAIM_COL].astype(str)
joined = joined[joined[CLAIM_COL].str.strip() != ""]

# Encode the query (first row's claim)
query_claim = joined.loc[0, CLAIM_COL]
query_vec = model.encode([query_claim], normalize_embeddings=True)

# Encode all claims
all_vecs = model.encode(joined[CLAIM_COL].tolist(), normalize_embeddings=True)

# Compute cosine similarity
sims = cosine_similarity(query_vec, all_vecs)[0]

# Add similarity column
joined["similarity_to_first"] = sims

# Sort
df_sorted = joined.sort_values("similarity_to_first", ascending=False).reset_index(drop=True)

# Save results
df_sorted.to_csv("similarity_first_vs_all.csv", index=False)

print("✅ Done. Saved to similarity_first_vs_all.csv")
print(df_sorted.head(10)[["patent_id", CLAIM_COL, "similarity_to_first"]])


✅ Done. Saved to similarity_first_vs_all.csv
  patent_id                                      cleaned_claim  \
0   8442755  1. A system for a motor vehicle including a mo...   
1   8442755  11. A system for a motor vehicle including a m...   
2   8442755  17. A method carried-out by a vehicular system...   
3   7193509  1. A door system for a motor vehicle, comprisi...   
4   9797181  5. A system for a vehicle, the system comprisi...   
5   8373550  22. A system for motorizing at least one door ...   
6   5008603  1. A system for controlling a position of fold...   
7   9797181  1. A system for a vehicle, the system comprisi...   
8   7761209  12. A vehicle, comprising: a vehicle body; a d...   
9   7438346  23. A vehicle, comprising: a door frame; a doo...   

   similarity_to_first  
0             1.000000  
1             0.845720  
2             0.745241  
3             0.702872  
4             0.701225  
5             0.700897  
6             0.693432  
7             0.690008  
8  

In [57]:
import pandas as pd
import numpy as np

df = pd.read_csv("..\\result tables\\check1.csv")
k = 10

retrieved_ids = []
found_itself_flags = []
retrieved_sims = []  # NEW: similarities for each retrieved ID

for i, row in df.iterrows():
    claim = row['claim']
    true_patent_id = str(row['patent_id']).strip()

    if pd.isna(claim) or not claim.strip():
        retrieved_ids.append("")
        found_itself_flags.append("N")
        retrieved_sims.append("")  # keep same length
        continue

    search_k = k * 5 
    query_embedding = model.encode([claim], normalize_embeddings=True)

    distances, indices = index.search(query_embedding.astype("float32"), search_k)
    
    top_k_metadata = metadata.iloc[indices[0]]
    all_ids = top_k_metadata['patent_id'].astype(str)
    all_sims = distances[0]

    # Keep unique IDs with their first similarity score
    unique_ids = []
    unique_sims = []
    seen = set()
    for pid, sim in zip(all_ids, all_sims):
        if pid not in seen:
            seen.add(pid)
            unique_ids.append(pid)
            unique_sims.append(f"{sim:.4f}")  # format to 4 decimals
        if len(unique_ids) == k:
            break

    retrieved_ids.append(" | ".join(unique_ids))
    retrieved_sims.append(" | ".join(unique_sims))
    found_itself_flags.append("Y" if true_patent_id in unique_ids else "N")

df['top 10 retrieved patents'] = retrieved_ids
df['similarities_top10'] = retrieved_sims  # NEW column
df['found itself (Y/N)'] = found_itself_flags

df.to_csv("..\\check2_with_retrieved_cos.csv", index=False)
print("Retrieval + similarity logging done.")


Retrieval + similarity logging done.
